In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import h5py
import pandas as pd
import numpy as np

from pathlib import Path
import re

import itertools
sns.set_style('whitegrid')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
# Import dataset
sc_control_regex = re.compile(r'^293T\.(?P<color>\w+)\.(?P<rep>\d)-rep_Single Cells')
n_gene_regex = re.compile(r'^293T\.(?P<condition>\d-gene(?:-no-rTTa)?(?:\.\w+)?)\.(?P<dox>.*)-dox\.(?P<rep>\d)-rep_Single Cells')
camp_regex = re.compile(r'^(?P<condition>[^\.]+)\.(?P<dox>.*)_dox\.(?P<camp>\d+)nM_camp\.(?P<rep>\d)(?:\(1\))?_Single Cells')
sc_controls = []
experimental = []
camp_dox_titration = []
for file in Path('../output/flow/2021.05.07-T2.0/').glob('*.csv'):
    sc_match = sc_control_regex.match(file.stem)
    exp_match = n_gene_regex.match(file.stem)
    if sc_match is not None:
        sc_controls.append(pd.read_csv(file))
        sc_controls[-1]['condition'] = sc_match.group('color')
        sc_controls[-1]['replicate'] = int(sc_match.group('rep'))
    elif exp_match is not None:
        experimental.append(pd.read_csv(file))
        experimental[-1]['condition'] = exp_match.group('condition')
        dox_str = exp_match.group('dox')
        if dox_str == '0':
            experimental[-1]['dox'] = 0.0
        if dox_str.startswith('E'):
            experimental[-1]['dox'] = 10**float(dox_str[1:])
        experimental[-1]['replicate'] = int(exp_match.group('rep'))

for file in Path('../output/flow/2021.05.22-T2.0_camp/').glob('*.csv'):
    match = camp_regex.match(file.stem)
    if match is not None:
        camp_dox_titration.append(pd.read_csv(file))
        camp_dox_titration[-1]['condition'] = match.group('condition')
        camp_dox_titration[-1]['camptothecin'] = match.group('camp')
        dox_str = match.group('dox')
        if dox_str == '0':
            camp_dox_titration[-1]['dox'] = 0.0
        if dox_str.startswith('e'):
            camp_dox_titration[-1]['dox'] = 10**float(dox_str[1:])
        camp_dox_titration[-1]['replicate'] = int(match.group('rep'))
sc_controls = pd.concat(sc_controls)
experimental = pd.concat(experimental)
camp_dox_titration = pd.concat(camp_dox_titration)

In [ ]:
# Count invalid Attune counts
print(np.sum((experimental['mRuby2-A'] < 0) | (experimental['eGFP-A'] < 0) | (experimental['iRFP670-A'] < 0)))
print(len(experimental))

In [ ]:
# Do postprocessing
experimental['mRuby2-normed'] = experimental['mRuby2-A'] / experimental['iRFP670-A']
experimental['eGFP-normed'] = experimental['eGFP-A'] / experimental['iRFP670-A']
gated = experimental[(experimental['iRFP670-A'] > 2e2) & np.array(['no-rTTa' not in x for x in experimental['condition']])]
camp_dox_titration['mRuby2-normed'] = camp_dox_titration['mRuby2-A'] / camp_dox_titration['iRFP670-A']
camp_dox_titration['eGFP-normed'] = camp_dox_titration['eGFP-A'] / camp_dox_titration['iRFP670-A']
gated_camp = camp_dox_titration[(camp_dox_titration['iRFP670-A'] > 2e2)]

In [ ]:
main_condition_palette = {
    '1-gene': '#808080',
    '2-gene.Tandem': '#A58245',
    '3-gene.Tandem': '#F1BE64',
    '2-gene.Divergent': '#18B7CE',
    '3-gene.Divergent': '#1DE2FF',
    '2-gene.Convergent': '#AC4140',
    '3-gene.Convergent': '#F85D5C',
}
light_background_palette = {k:v + '30' for k, v in main_condition_palette.items()}
plot_order = [
    '1-gene',
    '2-gene.Tandem', '3-gene.Tandem',
    '2-gene.Divergent', '3-gene.Divergent',
    '2-gene.Convergent', '3-gene.Convergent'
]

In [ ]:
sns.set_style('whitegrid')
for axis1, axis2 in itertools.combinations(['eGFP-A', 'mRuby2-A', 'iRFP670-A'], 2):
    sns.scatterplot(x=axis1, y=axis2,
        hue='condition', palette={
            'EGFP': '#26d334',
            'mRuby2': '#ef281a',
            'iRFP670': '#a8217b',
            'unstained': '#a8a4a7'
        }, alpha=.2, s=6,data=sc_controls)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim([10, max(sc_controls[axis1]) * 1.1])
    plt.ylim([10, max(sc_controls[axis2]) * 1.2])
    sns.despine()
    plt.legend(loc='upper right', bbox_to_anchor=(1.5,1))
    plt.savefig('../img/t2.0/sc_controls_{}_{}.png'.format(axis1,axis2), dpi=300)
    plt.show()

In [ ]:
sns.set_style('whitegrid')
plot_df = gated.copy()
plot_df['log iRFP670'] = np.log10(plot_df['iRFP670-A'])
sns.violinplot(x='condition', y='log iRFP670',
     palette=main_condition_palette, order=plot_order, data=plot_df)
ax = plt.gca()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.xlabel('')
plt.savefig('../img/t2.0/iRFP_expression_across_conditions.svg')
plt.show()
plot_df = gated.copy()
plt.title('Noise reporter expression, all induction conditions')
plot_df['log iRFP670'] = np.log10(plot_df['iRFP670-A'])

sns.violinplot(x='condition', y='log iRFP670',
     palette=main_condition_palette, order=plot_order, data=plot_df[plot_df['dox'] == 0])
ax = plt.gca()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
plt.xlabel('')
plt.title('Noise reporter expression, uninduced')
plt.savefig('../img/t2.0/iRFP_expression_across_conditions_dox_0.svg')
plt.show()

In [ ]:
sns.set_style('whitegrid')
plot_df = gated[gated['mRuby2-A'] > 1].copy()
plot_df['log mRuby2'] = np.log10(plot_df['mRuby2-A'])
plot_df['log normed mRuby2'] = np.log10(plot_df['mRuby2-normed'])
for col in ['log mRuby2', 'log normed mRuby2']:
    sns.violinplot(x='condition', y=col,
        palette=main_condition_palette, order=plot_order, data=plot_df[plot_df['dox'] == 0])
    ax = plt.gca()
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    plt.xlabel('')
    plt.title('Constitutive reporter output, uninduced')
    plt.savefig('../img/t2.0/{}_expression_0_dox.svg'.format(col))
    plt.show()

In [ ]:
sns.set_style('white')
plot_df = gated[(gated['mRuby2-A'] > 1) & (gated['eGFP-A'] > 1)].copy()
filter_set = plot_df['dox'].unique()[1::2]
plot_df = plot_df[plot_df['dox'].apply(lambda x: x == 0 or x in filter_set)]
plot_df['log mRuby2-A'] = np.log10(plot_df['mRuby2-A'])
plot_df['log mRuby2-N'] = np.log10(plot_df['mRuby2-normed'])
plot_df['mRuby2-N'] = plot_df['mRuby2-normed']
plot_df['log eGFP-A'] = np.log10(plot_df['eGFP-A'])
plot_df['log eGFP-N'] = np.log10(plot_df['eGFP-normed'])
plot_df['eGFP-N'] = plot_df['eGFP-normed']
for log_str, normed_str, gene_num, xlim, ylim in [
        ('', '-N', '2', (-.2, 6), (-.2, 20)),
        ('', '-N', '3', (-.2, 6), (-.2, 20)),
        ('log ', '-N', '2', None, None),
        ('log ', '-N', '3', None, None),
        ('log ', '-A', '2', None, None),
        ('log ', '-A', '3', None, None)
        ]:
    subsetted_df = plot_df[plot_df['condition'].apply(lambda x: x[0] == '1' or x[0] == gene_num)]
    subsetted_order = [x for x in plot_order if x[0] == '1' or x[0] == gene_num]
    kwargs = {}
    if xlim is not None:
        kwargs['xlim'] = xlim
    if ylim is not None:
        kwargs['ylim'] = ylim
    grid = sns.FacetGrid(subsetted_df, col='dox', row='condition',
            row_order=subsetted_order,
            margin_titles=True, despine=False, **kwargs)
    grid.map(sns.scatterplot,
        log_str + 'eGFP' + normed_str,
        log_str + 'mRuby2' + normed_str, color='#303030', alpha=0.4, s=10)
    grid.fig.subplots_adjust(wspace=0, hspace=0)
    for axis in grid.axes[0]:
        dox = float(axis.get_title().split('=')[1].strip())
        if dox == 0:
            axis.set_title('Uninduced')
        else:
            axis.set_title('10^{} induction'.format(np.log10(dox)))
    # Adjustment from https://cduvallet.github.io/posts/2018/11/facetgrid-ylabel-access
    for axis in grid.axes.flat:
        if axis.texts:
            txt = axis.texts[0]
            axis.text(*txt.get_unitless_position(),
                    txt.get_text().split('=')[1].strip(),
                    transform=axis.transAxes,
                    va='center')
            axis.texts[0].remove()

    for (row_val,col_val), ax in grid.axes_dict.items():
        ax.set_facecolor(light_background_palette[row_val])
    plt.savefig('../img/t2.0/induction_plot_{}{}_{}gene.png'.format(
        log_str.strip(), normed_str, gene_num
    ), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
sns.set_style('white')
plot_df = gated_camp[(gated_camp['mRuby2-A'] > 1) & (gated_camp['eGFP-A'] > 1)].copy()
filter_set = plot_df['dox'].unique()[1::2]
plot_df = plot_df[plot_df['dox'].apply(lambda x: x == 0 or x in filter_set)]
plot_df['log mRuby2-A'] = np.log10(plot_df['mRuby2-A'])
plot_df['log mRuby2-N'] = np.log10(plot_df['mRuby2-normed'])
plot_df['mRuby2-N'] = plot_df['mRuby2-normed']
plot_df['log eGFP-A'] = np.log10(plot_df['eGFP-A'])
plot_df['log eGFP-N'] = np.log10(plot_df['eGFP-normed'])
plot_df['eGFP-N'] = plot_df['eGFP-normed']

print(plot_df['dox'].unique())
print(plot_df['camptothecin'].unique())
print(plot_df['condition'].unique())
for log_str, normed_str, xlim, ylim in [
        ('', '-N', '2', (-.2, 6), (-.2, 20))
        ]:
    grid = sns.FacetGrid(plot_df[plot_df['condition'] == '2-gene-divergent'], col='dox', row='camptothecin',
                margin_titles=True, despine=False, **kwargs)
    grid.map(sns.scatterplot,
        log_str + 'eGFP' + normed_str,
        log_str + 'mRuby2' + normed_str, color='#303030', alpha=0.4, s=10)
    grid.fig.subplots_adjust(wspace=0, hspace=0)
    for axis in grid.axes[0]:
        dox = float(axis.get_title().split('=')[1].strip())
        if dox == 0:
            axis.set_title('Uninduced')
        else:
            axis.set_title('10^{} induction'.format(np.log10(dox)))
    # Adjustment from https://cduvallet.github.io/posts/2018/11/facetgrid-ylabel-access
    for axis in grid.axes.flat:
        if axis.texts:
            txt = axis.texts[0]
            axis.text(*txt.get_unitless_position(),
                    txt.get_text().split('=')[1].strip(),
                    transform=axis.transAxes,
                    va='center')
            axis.texts[0].remove()

    for (row_val,col_val), ax in grid.axes_dict.items():
        ax.set_facecolor(light_background_palette[row_val])
    plt.savefig('../img/t2.0/camp_plot.png'.format(
        log_str.strip(), normed_str, gene_num
    ), bbox_inches='tight', dpi=300)
    plt.show()